In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
--2024-07-03 23:11:43--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-03 23:11:43 (29.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [41]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""

    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm1(prompt):
    response = client.chat.completions.create(
        model='gemma2b',
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    
    return response.choices[0].message.content

In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [42]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [43]:
llm1("What's the formula for energy?")

NotFoundError: Error code: 404 - {'error': {'message': "model 'gemma2b' not found, try pulling it first", 'type': 'api_error', 'param': None, 'code': None}}

In [10]:
print(_)

 This message serves as an automated alert to verify the functionality of communication channels or systems. It indicates no actual testing scenario, just places before you for context purposes in system operations:

```python
# Placeholder code block simulating sending "This is a test" through various channels (e.g., email, SMS). In real scenarios replace with respective APIs calls/logic
def send_test_message(channels):
    message = "This is a test"
    
    for channel in channels:  # Iterate over different communication mediums or services such as EmailService(), PhoneCall(), etc.
        print("Sending '{}' through {}".format(message, str(channel)))
        try:
            result = channel.send_message(message)
            if not result['success']:  # In real implementation check the response or status of message delivery for each medium/service used
                raise Exception("Failed to send '{}' through {}: {}".format(message, str(channel), result['error']))
        except

In [5]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.5/124.5 kB 2.9 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 7.8 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.7/423.7 kB 11.3 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.4 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


---------------------------------------------------------------------------------------------------------------------------------

In [1]:
!pip install tika PyPDF2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:0018.1 MB/s eta 0:00:01
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32623 sha256=76b58ad53af7398ad89448ff485a219396ba7068bfa47af0d5f68637c0f9f3ea
  Stored in directory: /home/codespace/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [25]:
import PyPDF2
from tika import parser

# Open the PDF file
with open('The Oxford Dictionary of Quotations - 200 pages.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)

    # Specify the starting page
    start_page = 77

    # Extract text from the specified page
    page = reader.pages[start_page - 1]  # Page numbers are 0-based in PyPDF2
    page_text = page.extract_text()

# Use Tika to parse the text of the first page
parsed = parser.from_buffer(page_text)

print(parsed['content'])










 1.1 Peter Abelard 1079-1142
    O quanta qualia sunt illa sabbata,
    Quae semper celebrat superna curia.
    O what their joy and glory must be,
    Those endless sabbaths the blesséd ones see!
‘Hymnarius Paraclitensis’ bk. 1, pars altera ‘Hymni Diurni’ no. 29 ‘Sabbato. Ad Vesperas’ (translated by J. M. Neale, 1854)
 1.2 Dannie Abse 1923—
    I know the colour rose, and it is lovely,
    But not when it ripens in a tumour;    And healing greens, leaves and grass, so springlike,    In limbs that fester are not springlike.
‘Pathology of Colours’ (1968)
    So in the simple blessing of a rainbow,    In the bevelled edge of a sunlit mirror,    I have seen visible, Death’s artifact    Like a soldier’s ribbon on a tunic tacked.
‘Pathology of Colours’ (1968)
 1.3 Accius 170-c.86 B.C.
    Oderint, dum metuant.
    Let them hate, so long as they fear.
From ‘Atreus’, in Seneca ‘Dialogues’ bks. 3-5 ‘De Ira’ bk. 1, sect. 20, subsect. 4
 1.4 Goodman Ace 1899-1982
    TV—a clever contrac

In [4]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [6]:
pip install ollama


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
import ollama
response = ollama.chat(model='gemma2b', messages=[
  {
    'role': 'user',
    'content': "What's the formula for energy?",
    temperature=0.0
  },
])
print(response['message']['content'])

SyntaxError: ':' expected after dictionary key (1827518245.py, line 6)

In [26]:
text_content = parsed['content']
print(text_content)










 1.1 Peter Abelard 1079-1142
    O quanta qualia sunt illa sabbata,
    Quae semper celebrat superna curia.
    O what their joy and glory must be,
    Those endless sabbaths the blesséd ones see!
‘Hymnarius Paraclitensis’ bk. 1, pars altera ‘Hymni Diurni’ no. 29 ‘Sabbato. Ad Vesperas’ (translated by J. M. Neale, 1854)
 1.2 Dannie Abse 1923—
    I know the colour rose, and it is lovely,
    But not when it ripens in a tumour;    And healing greens, leaves and grass, so springlike,    In limbs that fester are not springlike.
‘Pathology of Colours’ (1968)
    So in the simple blessing of a rainbow,    In the bevelled edge of a sunlit mirror,    I have seen visible, Death’s artifact    Like a soldier’s ribbon on a tunic tacked.
‘Pathology of Colours’ (1968)
 1.3 Accius 170-c.86 B.C.
    Oderint, dum metuant.
    Let them hate, so long as they fear.
From ‘Atreus’, in Seneca ‘Dialogues’ bks. 3-5 ‘De Ira’ bk. 1, sect. 20, subsect. 4
 1.4 Goodman Ace 1899-1982
    TV—a clever contrac

In [31]:
def generate_prompt(query):
    prompt_template = """
You've to act like a Quote Extractor.The PROMPT says what action you have to take on the CONTEXT provided. 
Identify the pattern in which the quotes occur in the CONTEXT and extract them from the CONTEXT.
Perform the action only based on the CONTEXT and return None if the CONTEXT is irrrelevant to the PROMPT

PROMPT: {prompt}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    # for doc in search_results:
    #     context = context + f"prompt: {text_content}"
    context = context + f"{text_content}"
    prompt = prompt_template.format(prompt=query, context=context).strip()
    return prompt

q = generate_prompt("Extract all the quotes from the context provided")
print(q)

You've to act like a Quote Extractor.The PROMPT says what action you have to take on the CONTEXT provided. 
Identify the pattern in which the quotes occur in the CONTEXT and extract them from the CONTEXT.
Perform the action only based on the CONTEXT and return None if the CONTEXT is irrrelevant to the PROMPT

PROMPT: Extract all the quotes from the context provided

CONTEXT: 









 1.1 Peter Abelard 1079-1142
    O quanta qualia sunt illa sabbata,
    Quae semper celebrat superna curia.
    O what their joy and glory must be,
    Those endless sabbaths the blesséd ones see!
‘Hymnarius Paraclitensis’ bk. 1, pars altera ‘Hymni Diurni’ no. 29 ‘Sabbato. Ad Vesperas’ (translated by J. M. Neale, 1854)
 1.2 Dannie Abse 1923—
    I know the colour rose, and it is lovely,
    But not when it ripens in a tumour;    And healing greens, leaves and grass, so springlike,    In limbs that fester are not springlike.
‘Pathology of Colours’ (1968)
    So in the simple blessing of a rainbow,    In th

In [32]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [33]:
quotes = llm(q)

In [34]:
print(quotes)

 ["O quanta qualia sunt illa sabbata,", "Quae semper celebrat superna curia.", "", "", "I know the colour rose, and it is lovely,"..., "Let them hate, so long as they fear."]

The pattern in this CONTEXT involves quotes from different authors that express their thoughts or statements. However, some text appears to be a mix of Latin quotations and English prose with no clear separation between the two types of content within each author's passage. As per PROMPT instruction where we are asked only based on context provided, I extracted all parts surrounded by double quotes as they represent direct speech or significant statements from different authors in CONTEXT:

["O quanta qualia sunt illa sabbata,", "Quae semper celebrat superna curia.", "", "", "...Let them hate, so long as they fear."]
